# Advanced Python: BYONN
**B**uild **Y**our **O**wn **N**eural **N**etwork

<center>
<img src="../pictures/okeeffe_bird.jpg" style="width:480px;height:360px;">
<br>
<i>A Black Bird with Snow-Covered Red Hills (1946, Georgia O'Keeffe)</i>
</center>

In the last sessions, we will explore the basics of building a neural network in Python. The goal is to understand the framework and packages well enough so that you can go on and explore on you own. Here, we will start from the very basics -- **Object-Oriented Programming (OOP)**.

References:
- [Class Notes (CMU-15112) Object-Oriented Programming(OOP)](https://www.krivers.net/15112-f18/notes/notes-oop.html)

### Warm-up Exercises:
1. What is a `type` in Python? Give 3 examples of types.

Key idea: An `object` is a data structure that contains user-defined `properties` and `methods`. Now imagine in an alternative world, I want an object (class) `Bird` that has properties like `species`, `wing_span` and `color`. Maybe we even want to record where it is in the world (`x`, `y`)...

In [ ]:
class Bird(object):
    # all objects needs an __init__ method
    def __init__(self, species, wing_span, color):
        self.species = species
        self.wing_span = wing_span
        self.color = color

        # all the bird start at the origin
        self.x = 0
        self.y = 0

In [6]:
greatBlueHeron = Bird("Great Blue Heron", 2.0, "blue-gray")
chikadee = Bird("Chickadee", 0.3, "black-capped")

print(type(chikadee))
print(chikadee.species)
print(greatBlueHeron.wing_span)

<class '__main__.Bird'>
Chickadee
2.0


Suppose all `Bird` can `fly`. We will add a method `fly` to our `Bird` class that updates the position of the bird by a given distance in the x and y direction. We could even have the `Bird` report where it is in our world.

In [8]:
class Bird(object):
    # all objects needs an __init__ method
    def __init__(self, species, wing_span, color):
        self.species = species
        self.wing_span = wing_span
        self.color = color

        # all the bird start at the origin
        self.x = 0
        self.y = 0

    def fly(self, dx, dy):
        self.x += dx
        self.y += dy

    def where(self):
        print(f"{self.species} is at ({self.x}, {self.y})")
        return (self.x, self.y)

In [10]:
greatBlueHeron = Bird("Great Blue Heron", 2.0, "blue-gray")

for i in range(4):
    greatBlueHeron.fly(10, 2)
    greatBlueHeron.where()

Great Blue Heron is at (10, 2)
Great Blue Heron is at (20, 4)
Great Blue Heron is at (30, 6)
Great Blue Heron is at (40, 8)


You could even define a subclass of a `Bird` (let's say `Penguin`) that has a pre-defined `fly` function that does not change the y position of the instances (because... they don't fly://).

In [14]:
class Penguin(Bird):
    def __init__(self, wing_span):
        super().__init__("Penguin", wing_span, "black and white")

    def fly(self, dx):
        # Penguins don't fly, so we don't change the y position
        self.x += dx
        # self.y += dy  # Uncommenting this line would allow penguins to "fly"]

    def isBigger(self, other):
        return self.wing_span > other.wing_span

In [15]:
adelie = Penguin(0.6)
emperor = Penguin(1.0)

print("is adelie bigger than emperor?", adelie.isBigger(emperor))

is adelie bigger than emperor? False


Wait... This feels familiar? Where have we seen *OOP* in our sessions before?


### Back to Neural Networks
Of course, now with the knowledge of OOP, we could construct our own neural network, from scratch. OR, we could use a framework like TensorFlow or PyTorch that already implements these concepts for us.

In [16]:
import numpy as np

In [19]:
class Neuron(object):
    def __init__(self, weights, bias, activation):
        self.weights = weights
        self.bias = bias
        self.activation = activation

    def compute(self, inputs):
        z = sum(w * i for w, i in zip(self.weights, inputs)) + self.bias
        # python can take a function as an input
        return self.activation(z)

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x)) 

def relu(x):
    return max(0, x)

def linear(x):
    return x

In [20]:
neuron1 = Neuron(weights=[0.5, -0.2], bias=0.1, activation=sigmoid)
neuron1.compute([1.0, 2.0])

np.float64(0.549833997312478)

It would be troublesome to write out code to initiate each of the neurons; not to mention that most of the times, the neurons themselves are the same within a layer. Therefore, with `pytorch`, the smallest "units" are often thought of as `layers`. 

In [22]:
import torch
import torch.nn as nn

For example, a `Linear` layer in `pytorch` is a layer that has a set of weights and biases, and applies a linear transformation to the input*. The `Linear` layer can be thought of as a collection of neurons that are connected to the input.

*Formally, for incoming data $x$, the output is given by $$y=xA^T + b $$

where $A$ is the weight matrix and $b$ is the bias vector.



In [24]:
l1 = nn.Linear(10, 20) # in_feature, out_feature

inputs = torch.rand((5, 10)) # batch/sample size, in_feature
outputs = l1(inputs) # outputs will be of shape (5, 20)

print(outputs.shape)

torch.Size([5, 20])


A single layer does not carry you far. To define a feedfoward neural network, we typically need to stack multiple layers. In `pytorch`, we can define any neural network by defining a class that inherits from `nn.Module` (base class for all neural network modules, [doc](https://docs.pytorch.org/docs/stable/generated/torch.nn.Module.html)).

In [ ]:
class simpleNN(nn.Module): # inherit from nn.Module
    def __init__(self):
        super(simpleNN, self).__init__()
        # 2 linear layers
        self.layer1 = nn.Linear(10, 20)
        self.layer2 = nn.Linear(20, 5)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.relu(x)  # using ReLU activation
        x = self.layer2(x)
        return x